Tested only on google colab. If there is an error with numpy you should restart the kernel. It is okay

In [3]:
!pip install norse tonic --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 7.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import tonic.transforms as transforms
import norse
import tonic
from tonic import DiskCachedDataset
from tonic.collation import PadTensors

save_path = "./data/nmnist"

# Hyperparameters
dtype = torch.float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 64

# Data Loading
frame_transform = transforms.Compose([
    transforms.ToFrame(
        sensor_size=tonic.datasets.NMNIST.sensor_size,
        time_window=60000
    )
])

trainset = DiskCachedDataset(
    tonic.datasets.NMNIST(train=True, save_to=save_path),
    transform=frame_transform,
    cache_path="./cache/nmnist/train"
)

testset = DiskCachedDataset(
    tonic.datasets.NMNIST(train=False, save_to=save_path),
    transform=frame_transform,
    cache_path="./cache/nmnist/test"
)


train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True,
                          collate_fn=PadTensors(batch_first=False))

test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False,
                         collate_fn=PadTensors(batch_first=False))

  0%|          | 0/1011893601 [00:00<?, ?it/s]

Extracting ./data/nmnist/NMNIST/train.zip to ./data/nmnist/NMNIST


  0%|          | 0/169674850 [00:00<?, ?it/s]

Extracting ./data/nmnist/NMNIST/test.zip to ./data/nmnist/NMNIST


In [3]:
norse_model = norse.torch.from_nir("nir_model.nir", dt=0.0001)

In [ ]:
def apply(data):
    """
    apply an input data batch to the norse model
    """
    state = None
    hid_rec = []
    out = []

    for i, t in enumerate(data):
        z, state = norse_model(t.flatten(1), state)
        out.append(z)
        hid_rec.append(state)
    spk_out = torch.stack(out)
    return spk_out, hid_rec

In [ ]:
def measure_accuracy2(model, dataloader):
  with torch.no_grad():
    running_length = 0
    running_accuracy = 0

    for data, targets in iter(dataloader):
      spk_rec, _ = model(data)
      spike_count = spk_rec.sum(0)
      _, max_spike = spike_count.max(1)

      num_correct = (max_spike == targets).sum()

      running_length += len(targets)
      running_accuracy += num_correct

    accuracy = (running_accuracy / running_length)

    return accuracy.item()

In [6]:
measure_accuracy2(apply, test_loader)

0.8906000256538391